# Supplementary figure about the scaling parameter $\Lambda$
Show that we can make BioPCA and IBCM equivalent. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json
pj = os.path.join

In [ ]:
do_save_plots = True
# Resources
root_dir = pj("..", "..", "..")
data_folder = pj(root_dir, "results", "for_plots")
panels_folder = "panels/"
params_folder = pj(root_dir, "results", "common_params")

In [ ]:
# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full24 = np.asarray(json.load(f))
# Here, 32 neurons, need to make a new palette with same parameters
neuron_colors_full = np.asarray(sns.husl_palette(n_colors=32, h=0.01, s=0.9, l=0.4, as_cmap=False))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

# Plotting functions

In [ ]:
def plot_ystats(stats_file):
    # Compare IBCM and PCA as a function of Lambda
    # Plot y statistics vs lambda
    models = ["ibcm", "biopca"]
    models_short = {"ibcm":"ibcm", "biopca":"pca"}
    
    # Get expected Lambda at which numerical instabilities arise
    lambda_limits = {a:stats_file[f"lambda_limits_{a}"] for a in models_short.values()}
    
    # First, plots of statistics vs lambda
    fig, axes = plt.subplots(1, 3, sharex=True)
    fig.set_size_inches(plt.rcParams["figure.figsize"][0]*2, plt.rcParams["figure.figsize"][1])
    axes = axes.flatten()
    for m in models[::-1]:
        mshort = models_short[m]
        all_stats = stats_file[m]
        lambd_axis = stats_file[f"lambda_ranges_{mshort}"]
        mask = all_stats > 5.0  # Clip excessively large values
        masked_stats = np.ma.masked_array(all_stats, mask)
        lambd_0 = stats_file[f"lambda_zeros_{mshort}"][0]
        lambd_0 = 1.0
        for i in range(all_stats.shape[1]):
            axes[i].plot(lambd_axis / lambd_0, masked_stats[:, i],
                label=model_nice_names.get(m, m),
                color=model_colors.get(m), lw=2.0
            )
            axes[i].axvline(lambda_limits[mshort] / lambd_0, ls="--", color=model_colors.get(m), lw=1.0)
    # Labeling the graphs, etc.
    stat_names = ["Mean", "Variance", "Third moment"]
    for i in range(all_stats.shape[1]):
        ax = axes[i]
        axes[i].set_title(stat_names[i])
        axes[i].set_xlabel(r"Scale of $M$, $\Lambda$")
        #axes[i].set_xlabel(r"Scale of $M$, $\Lambda$")
        axes[i].set_ylabel(stat_names[i] + r" of $\| \mathbf{y} \|$")
        axes[i].set_xscale("log")
    axes[0].legend(frameon=True, loc="upper left")
    return fig, axes

In [ ]:
# Second, plots of Jaccard median and mean similarities, for each new conc.
def plot_jaccards(jacs_file, stats_file):
    models = ["ibcm", "biopca"]
    models_short = {"ibcm":"ibcm", "biopca":"pca"}
    
    # Get expected Lambda at which numerical instabilities arise
    lambda_limits = {a:stats_file[f"lambda_limits_{a}"] for a in models_short.values()}
    new_concs = jacs_file["new_concs"]
    n_new_concs = len(new_concs)
    # One plot per new odor concentration
    fig, axes = plt.subplots(1, n_new_concs, sharex=True, sharey=True)
    fig.set_size_inches(plt.rcParams["figure.figsize"][0]*2, plt.rcParams["figure.figsize"][1])
    # Top row: median, bottom row: mean
    # Really, just show the mean, that's enough
    for m in models[::-1]:  # Plot IBCM last
        mshort = models_short[m]
        lambd_axis = stats_file[f"lambda_ranges_{mshort}"]
        all_jacs = jacs_file[m]
        median_jacs = np.median(all_jacs, axis=(1, 2, 4))
        mean_jacs = np.mean(all_jacs, axis=(1, 2, 4))
        stdev_jacs = np.std(all_jacs, axis=(1, 2, 4))
        lambd_0 = stats_file[f"lambda_zeros_{mshort}"][0]
        lambd_0 = 1.0
        for i in range(n_new_concs):
            axes[i].plot(lambd_axis / lambd_0, mean_jacs[:, i],
                        label=model_nice_names.get(m, m),
                        color=model_colors.get(m), lw=2.0)
            axes[i].fill_between(lambd_axis / lambd_0, 
                    np.clip(mean_jacs[:, i] - stdev_jacs[:, i], a_min=0.0, a_max=np.inf), 
                    mean_jacs[:, i] + stdev_jacs[:, i], color=model_colors.get(m), alpha=0.3)
            axes[i].axvline(lambda_limits[mshort] / lambd_0, ls="--", 
                            color=model_colors.get(m), ymax=0.8)
    # Labeling the graphs, etc.
    new_concs_scaled = [0.5, 1.0]
    for i in range(n_new_concs):
        axes[i].set_title(r"New conc. = {:.1f} $\langle c \rangle$".format(new_concs_scaled[i]))
        axes[i].set_xlabel(r"Scale of $M$, $\Lambda$")
        axes[i].set_ylabel("Mean Jaccard similarity")
        axes[i].set_xscale("log")
        arrowprops = {"headwidth":2.5, "width":0.3, "color":"grey", "headlength":4.0}
        xytext = (0.85, 0.98)
        lbl = "Numerical\ninstability"
        axes[i].annotate(lbl, xytext=xytext, xy=(0.55, 0.2),
            color="grey", xycoords="axes fraction", va="top", ha="right", size=6.0, 
            arrowprops=arrowprops
        )
        axes[i].annotate(lbl, xytext=xytext, xy=(0.92, 0.2), color=(0, 0, 0, 0),
            xycoords="axes fraction", va="top", ha="right", size=6.0, arrowprops=arrowprops
        )
        
    for i in range(1, n_new_concs):
        axes[i].yaxis.set_tick_params(which='both', labelleft=True)
    axes[0].legend(frameon=True, loc="upper left")
    return fig, axes

# Panels A-B: Background inhibition and odor recognition results
Don't scale with respect to $\Lambda_0$, show the absolute $\Lambda$ parameter values, instead of trying to line up the models. 

In [ ]:
# Load values to plot
# ystats
dim_choice1 = 25  # or 300
activ_fct = "identity"
ystats_file1 = np.load(pj(data_folder, f"ystats_lambda_{dim_choice1}_{activ_fct}.npz"))
# Contents of the above file:
# lambda_limits_ibcm=lambda_limits["ibcm"],
# lambda_limits_pca=lambda_limits["biopca"],
# lambda_zeros_ibcm=lambda_zeros["ibcm"],
# lambda_zeros_pca=lambda_zeros["biopca"], 
# lambda_ranges_ibcm=lambda_ranges["ibcm"],
# lambda_ranges_pca=lambda_ranges["biopca"],
# ibcm, biopca

jaccards_file1 = np.load(pj(data_folder, f"jaccard_similarities_lambda_{dim_choice1}_{activ_fct}.npz"))
# Contents of the above file:
# ibcm, biopca, new_concs

In [ ]:
fig, axes = plot_ystats(ystats_file1)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, f"supfig_lambda_y_stats_vs_{dim_choice1}.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, axes = plot_jaccards(jaccards_file1, ystats_file1)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, f"supfig_lambda_jaccards_vs_{dim_choice1}.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Same panels, but for $N_S = 300$
Alternate figure. 

In [ ]:
# Load values to plot
# ystats
dim_choice2 = 300  # or 300
activ_fct = "identity"
ystats_file2 = np.load(pj(data_folder, f"ystats_lambda_{dim_choice2}_{activ_fct}.npz"))

jaccards_file2 = np.load(pj(data_folder, f"jaccard_similarities_lambda_{dim_choice2}_{activ_fct}.npz"))
# ibcm, biopca, new_concs

In [ ]:
fig, axes = plot_ystats(ystats_file2)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, f"supfig_lambda_y_stats_vs_{dim_choice2}.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, axes = plot_jaccards(jaccards_file2, ystats_file2)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, f"supfig_lambda_jaccards_vs_{dim_choice2}.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()